**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# ⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️
#
# Outdated and broken

will be updated, stay tuned

#
# ⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️⚠️

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/5305a470ca0260247560b4f94daf68ed62d4a514/85ceb/img/logos/opa-no-text-color.png" width="20%"/>
</div>

## Open Policy Agent

requires a running ingress, run `minikube addons enable ingress` to start an ingress 

following the [instructions in the tutorial](https://www.openpolicyagent.org/docs/latest/kubernetes-tutorial/#1-start-kubernetes-recommended-admisson-controllers-enabled)

In [5]:
kubectl get pods -A | grep --color=always -E '$|ingress' 

NAMESPACE       NAME                                        READY   STATUS      RESTARTS   AGE
default         acid-acid-mypg-clone-0                      1/1     Running     0          3h48m
default         acid-mypg-0                                 1/1     Running     0          3h54m
default         acid-mypg-1                                 1/1     Running     0          4h4m
default         acid-mypg-2                                 1/1     Running     0          4h4m
default         acid-mypg-yaml-0                            1/1     Running     0          179m
default         acid-mypg-yaml-1                            1/1     Running     0          179m
default         acid-mypg-yaml-2                            1/1     Running     0          179m
default         postgres-operator-58b75c5587-k7clb          1/1     Running     0          4h22m
default         postgres-operator-ui-58644cfcff-bx6xg       1/1     Running     0          4h21m
default         sleep-56bc8d768-tj2hm

In [6]:
kubectl create namespace opa


namespace/opa created


In [7]:
pwd

/minikube-host/notebooks/training-kubernetes-security


In [8]:
mkdir -p opa
cd opa

## Generate the certificate 
create a certificate authority (CA) and certificate/key pair 

In [9]:
openssl genrsa -out ca.key 2048
openssl req -x509 -new -nodes -key ca.key -days 100000 -out ca.crt -subj "/CN=admission_ca"

Generating RSA private key, 2048 bit long modulus (2 primes)
........................................................................................................................................+++++
.....................................................................................................................................................................+++++
e is 65537 (0x010001)


generate the TLS key and certificate for OPA

In [10]:
cat >server.conf <<EOF
[req]
req_extensions = v3_req
distinguished_name = req_distinguished_name
prompt = no
[req_distinguished_name]
CN = opa.opa.svc
[ v3_req ]
basicConstraints = CA:FALSE
keyUsage = nonRepudiation, digitalSignature, keyEncipherment
extendedKeyUsage = clientAuth, serverAuth
subjectAltName = @alt_names
[alt_names]
DNS.1 = opa.opa.svc
EOF

In [11]:
openssl genrsa -out server.key 2048
openssl req -new -key server.key -out server.csr -subj "/CN=opa.opa.svc" -config server.conf
openssl x509 -req -in server.csr -CA ca.crt -CAkey ca.key -CAcreateserial -out server.crt -days 100000 -extensions v3_req -extfile server.conf

Generating RSA private key, 2048 bit long modulus (2 primes)
..................+++++
............................+++++
e is 65537 (0x010001)
Signature ok
subject=CN = opa.opa.svc
Getting CA Private Key


In [45]:
kubectl create secret -n opa tls opa-server --cert=server.crt --key=server.key

error: failed to create secret secrets "opa-server" already exists


: 1

# AdmissionController
create config and deploy an admission controller

In [44]:
kubectl apply -f - <<EOF
# Grant OPA/kube-mgmt read-only access to resources. This lets kube-mgmt
# replicate resources into OPA so they can be used in policies.
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: opa-viewer
roleRef:
  kind: ClusterRole
  name: view
  apiGroup: rbac.authorization.k8s.io
subjects:
- kind: Group
  name: system:serviceaccounts:opa
  apiGroup: rbac.authorization.k8s.io
---
# Define role for OPA/kube-mgmt to update configmaps with policy status.
kind: Role
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  namespace: opa
  name: configmap-modifier
rules:
- apiGroups: [""]
  resources: ["configmaps"]
  verbs: ["update", "patch"]
---
# Grant OPA/kube-mgmt role defined above.
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  namespace: opa
  name: opa-configmap-modifier
roleRef:
  kind: Role
  name: configmap-modifier
  apiGroup: rbac.authorization.k8s.io
subjects:
- kind: Group
  name: system:serviceaccounts:opa
  apiGroup: rbac.authorization.k8s.io
---
kind: Service
apiVersion: v1
metadata:
  name: opa
  namespace: opa
spec:
  selector:
    app: opa
  ports:
  - name: https
    protocol: TCP
    port: 443
    targetPort: 443
---
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: opa
  namespace: opa
  name: opa
spec:
  replicas: 1
  selector:
    matchLabels:
      app: opa
  template:
    metadata:
      labels:
        app: opa
      name: opa
    spec:
      containers:
        # WARNING: OPA is NOT running with an authorization policy configured. This
        # means that clients can read and write policies in OPA. If you are
        # deploying OPA in an insecure environment, be sure to configure
        # authentication and authorization on the daemon. See the Security page for
        # details: https://www.openpolicyagent.org/docs/security.html.
        - name: opa
          image: openpolicyagent/opa:0.12.2
          args:
            - "run"
            - "--server"
            - "--tls-cert-file=/certs/tls.crt"
            - "--tls-private-key-file=/certs/tls.key"
            - "--addr=0.0.0.0:443"
            - "--addr=http://127.0.0.1:8181"
          volumeMounts:
            - readOnly: true
              mountPath: /certs
              name: opa-server
        - name: kube-mgmt
          image: openpolicyagent/kube-mgmt:0.8
          args:
            - "--replicate-cluster=v1/namespaces"
            - "--replicate=extensions/v1beta1/ingresses"
      volumes:
        - name: opa-server
          secret:
            secretName: opa-server
---
kind: ConfigMap
apiVersion: v1
metadata:
  name: opa-default-system-main
  namespace: opa
data:
  main: |
    package system

    import data.kubernetes.admission

    main = {
      "apiVersion": "admission.k8s.io/v1beta1",
      "kind": "AdmissionReview",
      "response": response,
    }

    default response = {"allowed": true}

    response = {
        "allowed": false,
        "status": {
            "reason": reason,
        },
    } {
        reason = concat(", ", admission.deny)
        reason != ""
    }
EOF

clusterrolebinding.rbac.authorization.k8s.io/opa-viewer unchanged
role.rbac.authorization.k8s.io/configmap-modifier unchanged
rolebinding.rbac.authorization.k8s.io/opa-configmap-modifier unchanged
service/opa unchanged
deployment.apps/opa created
configmap/opa-default-system-main unchanged


Exclude opa and kube-system namespaces

In [14]:
kubectl label ns kube-system openpolicyagent.org/webhook=ignore
kubectl label ns opa openpolicyagent.org/webhook=ignore


namespace/kube-system labeled
namespace/opa labeled


In [15]:
ls

ca.crt  ca.key  ca.srl  server.conf  server.crt  server.csr  server.key


In [47]:
kubectl apply -f - <<EOF
kind: ValidatingWebhookConfiguration
apiVersion: admissionregistration.k8s.io/v1beta1
metadata:
  name: opa-validating-webhook
webhooks:
  - name: validating-webhook.openpolicyagent.org
    namespaceSelector:
      matchExpressions:
      - key: openpolicyagent.org/webhook
        operator: NotIn
        values:
        - ignore
    rules:
      - operations: ["CREATE", "UPDATE"]
        apiGroups: ["*"]
        apiVersions: ["*"]
        resources: ["*"]
    clientConfig:
      caBundle: $(cat ca.crt | base64 | tr -d '\n')
      service:
        namespace: opa
        name: opa
EOF

validatingwebhookconfiguration.admissionregistration.k8s.io/opa-validating-webhook configured


In [21]:
cat  > repo-allowlist.rego <<EOF
package kubernetes.admission                                                # line 1

deny[msg] {                                                                 # line 2
  input.request.kind.kind == "Pod"                                          # line 3
  image := input.request.object.spec.containers[_].image                    # line 4
  not startswith(image, "gcr.io/")                                       # line 5
  msg := sprintf("image fails to come from trusted registry: %v", [image])  # line 6
}
EOF
ls

ca.crt  ca.srl               server.conf  server.csr
ca.key  repo-allowlist.rego  server.crt   server.key


In [48]:
kubectl get cm -n opa

NAME                      DATA   AGE
kube-root-ca.crt          1      15m
opa-default-system-main   1      3m32s
repo-allowlist            1      10m


In [22]:
kubectl create configmap -n opa repo-allowlist --from-file=repo-allowlist.rego

configmap/repo-allowlist created


In [54]:
kubectl run -n default blocklisted --image nginx --port=80

pod/blocklisted created


In [24]:
kubectl logs -n opa -l app=opa -c opa

  "level": "info",
  "msg": "Sent response.",
  "req_id": 61,
  "req_method": "GET",
  "req_path": "/health",
  "resp_bytes": 2,
  "resp_duration": 1.45354,
  "resp_status": 200,
  "time": "2021-12-02T13:36:59Z"
}


In [56]:
kubectl get pods -n default

NAME                                    READY   STATUS    RESTARTS   AGE
acid-acid-mypg-clone-0                  1/1     Running   0          4h4m
acid-mypg-0                             1/1     Running   0          4h11m
acid-mypg-1                             1/1     Running   0          4h21m
acid-mypg-2                             1/1     Running   0          4h21m
acid-mypg-yaml-0                        1/1     Running   0          3h16m
acid-mypg-yaml-1                        1/1     Running   0          3h16m
acid-mypg-yaml-2                        1/1     Running   0          3h16m
allowlisted                             1/1     Running   0          18s
blocklisted                             1/1     Running   0          7s
postgres-operator-58b75c5587-k7clb      1/1     Running   0          4h38m
postgres-operator-ui-58644cfcff-bx6xg   1/1     Running   0          4h38m
sleep-56bc8d768-tj2hm                   1/1     Running   0          23h


In [52]:
kubectl run -n default allowlisted  --labels="opa=good" --image=gcr.io/google-samples/node-hello:1.0  --port=8080

pod/allowlisted created


In [53]:
kubectl delete -n default pod blocklisted

Error from server (NotFound): pods "blocklisted" not found


: 1

In [51]:
kubectl delete -n default pod allowlisted

pod "allowlisted" deleted


In [ ]:
kubectl config use-context minikube

In [ ]:
kubectl get pods -n default

In [ ]:
kubectl delete namespace opa

In [ ]:
kubectl get pods -A

In [30]:
kubectl logs -n opa -l app=opa -c opa

  "level": "info",
  "msg": "Sent response.",
  "req_id": 95,
  "req_method": "GET",
  "req_path": "/health",
  "resp_bytes": 2,
  "resp_duration": 0.359101,
  "resp_status": 200,
  "time": "2021-12-02T13:38:09Z"
}


In [35]:
kubectl get all -n opa

NAME                       READY   STATUS    RESTARTS   AGE
pod/opa-79558b79f6-v5n95   2/2     Running   0          4m57s

NAME          TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/opa   ClusterIP   10.108.19.249   <none>        443/TCP   4m57s

NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/opa   1/1     1            1           4m57s

NAME                             DESIRED   CURRENT   READY   AGE
replicaset.apps/opa-79558b79f6   1         1         1       4m58s


In [43]:
kubectl get deployment -o yaml

apiVersion: v1
items:
- apiVersion: apps/v1
  kind: Deployment
  metadata:
    annotations:
      deployment.kubernetes.io/revision: "1"
      meta.helm.sh/release-name: postgres-operator
      meta.helm.sh/release-namespace: default
    creationTimestamp: "2021-12-02T09:10:09Z"
    generation: 1
    labels:
      app.kubernetes.io/instance: postgres-operator
      app.kubernetes.io/managed-by: Helm
      app.kubernetes.io/name: postgres-operator
      helm.sh/chart: postgres-operator-1.7.1
    name: postgres-operator
    namespace: default
    resourceVersion: "46491"
    uid: bfe9ccd0-bdb3-4b35-8c9e-235b043ec5c0
  spec:
    progressDeadlineSeconds: 600
    replicas: 1
    revisionHistoryLimit: 10
    selector:
      matchLabels:
        app.kubernetes.io/instance: postgres-operator
        app.kubernetes.io/name: postgres-operator
    strategy:
      rollingUpdate:
        maxSurge: 25%
        maxUnavailable: 25%
      type: RollingUpdate
    template:
      metadata:
        annota